In [1]:
%pip install osmnx
%pip install plotly_express
%pip install -r requirements.txt


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import osmnx as ox
import networkx as nx
from geopy.geocoders import Nominatim
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString
import plotly_express as px
import time

In [5]:
def geocode_with_retry(address, geolocator, max_retries=20, timeout=10):
    retries = 0
    while retries < max_retries:
        try:
            location = geolocator.geocode(address, timeout=timeout)
            return location
        except Exception:
            retries += 1
            print(f"Geocoder timed out, retrying... ({retries}/{max_retries})")
            time.sleep(1)
    return None

In [121]:
# %%writefile utils/Node_3.py
# %load utils/Node.py
class Node ():
    def __init__(self, state, value, operators=[], operator=None, parent=None, objective=None):
        self.state = state
        self.value = value
        self.children = []
        self.parent = parent
        self.operator = operator
        self.objective = objective
        self.level = 0
        self.operators = operators

    def add_child(self, value, state, operator):
        node = type(self)(value=value, state=state, operator=operator, parent=self, operators=self.operators)
        add_node_child(node)
        return node

    def add_node_child(self, node):
        node.level = node.parent.level + 1
        self.children.append(node)
        return node

  #Devuelve todos los estados según los operadores aplicados
    def get_childrens(self):
        children = []
        for i in range(len(self.operators)):
            state = self.get_state(i)
            if (self.is_ancestor(state)):
                children.append(None)
            else:
                children.append(state)
        
        return children
        # return [
        #     self.get_state(i)
        #       if not self.is_ancestor(self.get_state(i))
        #         else None 
        #         for i, op in enumerate(self.operators)
        #     ]


    def get_state(self, index):
        pass

    def is_ancestor(self, state):
        n = self
        while n is not None and n.state != state:
            n = n.parent
        return n is not None

    def path_to_objective(self):
        n = self
        result = []
        while n is not None:
            result.append(n)
            n = n.parent
        return result

    def heuristic(self):
        return 0

    def cost(self):
        return 1

    def f(self):
        return self.cost() + self.heuristic()

    def __eq__(self, other):
        return self.state == other.state

    def __lt__(self, other):
        return self.f() < other.f()


In [59]:
#%%writefile utils/Tree_3.py
class Tree ():
  def __init__(self, root, operators):
    self.root = root
    self.operators = operators

  def print_path(self, n):
    path = n.path_to_objective()
    for i in path:
        if i.operator is not None:
            print(f'operador:  {self.operators[node.operator]} \t estado: {node.state}')
        else:
            print(f' {node.state}')
    
    return path

    # stack = n.path_to_objective()
    # path = stack.copy()
    # while len(stack) != 0:
    #     node = stack.pop()
    #     if node.operator is not None:
    #         print(f'operador:  {self.operators[node.operator]} \t estado: {node.state}')
    #     else:
    #         print(f' {node.state}')
    # return path

  def reinit_root(self):
    self.root.operator = None
    self.root.parent = None
    self.root.objective = None
    self.root.children = []
    self.root.level = 0

  ## Primero a lo ancho
  def breadth_first(self,endState):
    self.reinit_root()
    pq = queue.Queue()
    pq.put(self.root)
    while not pq.empty():
      node = pq.get()
      children = node.get_childrens()
      for i, child in enumerate(children):
        if child is not None:
          newChild = node.add_child(value=node.value + '-' + str(i), state=child, operator=i)
          pq.put(newChild)
          if endState == child:
            return newChild

  ## Primero en profundidad
  def depht_first(self, endState):
    self.reinit_root()
    pq = []
    pq.append(self.root)
    while len(pq) > 0:
      node = pq.pop()
      if (node.parent is not None):
        node.parent.add_node_child(node)
      children = node.get_childrens()
      temp = []
      for i, child in enumerate(children):
        if child is not None:
          newChild = type(self.root)(value=node.value + '-' + str(i), state=child, operator=i, parent=node, operators=node.operators)
          temp.append(newChild)
          if endState == child:
            node.add_node_child(newChild)
            return newChild
      #Adicionar los hijos en forma inversa para que salga primero el primero que se adicionó
      temp.reverse()
      for e in temp:
        pq.append(e)

  ## Costo uniforme
  def uniform_cost(self, endState):
    self.reinit_root()
    pq = queue.PriorityQueue()
    pq.put((self.root.cost(), self.root))
    while not pq.empty():
      node = pq.get()[1]
      children = node.get_childrens()
      for i, child in enumerate(children):
        if child is not None:
          newChild = node.add_child(value=node.value + '-' + str(i), state=child, operator=i)
          pq.put((newChild.cost(), newChild))
          if endState == child:
            return newChild

  ## Primero el mejor
  def best_first(self, endState):
    self.reinit_root()
    pq = queue.PriorityQueue()
    pq.put((self.root.heuristic(), self.root))
    while not pq.empty():
      node = pq.get()[1]
      children = node.get_childrens()
      for i, child in enumerate(children):
        if child is not None:
          newChild = node.add_child(value=node.value + '-' + str(i), state=child, operator=i)
          pq.put((newChild.heuristic(), newChild))
          if endState == child:
            return newChild

  ## A*
  def astar(self, endState):
    self.reinit_root()
    pq = queue.PriorityQueue()
    pq.put((self.root.f(), self.root))
    while not pq.empty():
      node = pq.get()[1]
      children = node.get_childrens()
      for i, child in enumerate(children):
        if child is not None:
          newChild = node.add_child(value=node.value + '-' + str(i), state=child, operator=i)
          pq.put((newChild.f(), newChild))
          if endState == child:
            return newChild

  ## Método para dibujar el árbol
  def draw(self,path):
    graph = pydot.Dot(graph_type='graph')
    nodeGraph = pydot.Node(str(self.root.state) + "-" + str(0),
                          label=str(self.root.state), 
                          shape ="circle",
                          style="filled",
                          fillcolor="red")
    graph.add_node(nodeGraph)
    path.pop()
    return self.draw_tree_rec(self.root, nodeGraph, graph, 0, path.pop(), path)

  ## Método recursivo para dibujar el árbol
  def draw_tree_rec(self, r, rootGraph, graph, i, topPath, path):
    if r is not None:
      children = r.children
      for j,child in enumerate(children):
        i = i + 1
        color="white"
        if topPath.value == child.value:
          if len(path) > 0 : topPath = path.pop()
          color='red'
        c = pydot.Node(child.value, label=str(child.state) + r"\n\n" + "f=" + str(child.f()),
                      shape ="circle",
                      style="filled",
                      fillcolor=color)
        graph.add_node(c)
        graph.add_edge(pydot.Edge(rootGraph, c,
                                  label=str(child.operator) + '(' + str(child.cost()) + ')'))
        graph = self.draw_tree_rec(child, c, graph, i, topPath, path)  # recursive call
      return graph
    else:
      return graph


In [125]:
import pandas as pd
class Location(Node):
    # static data
    data=None
    def __init__(self, data: pd.DataFrame = None, **kwargs):
        super(Location, self).__init__(**kwargs)
        if data is not None:
            Location.data = data
        self.operators = self.get_operators(self.state)
    
    def get_operators(self, state):
        return [1, 2, 3, 4, 5]

    def get_state(self, index):
        return self.operators[index]



        

In [126]:
location = Location(state="A", value=1)

print(location.operators)
print(Location.data)
print(location.get_childrens())
print(location.operators[1])

[1, 2, 3, 4, 5]
None
[1, 2, 3, 4, 5]
2
